### Coding the Bose-Hubbard Hamiltonian with QuSpin

The purpose of this tutorial is to teach the interested user to construct bosonic Hamiltonians using QuSpin. To this end, below we focus on the Bose-Hubbard model (BHM) of a 1d chain. The Hamiltonian is
$$ H = -J\sum_{j=0}^{L-1}(b^\dagger_{j+1}b_j + \mathrm{h.c.})-\mu\sum_{j=0}^{L-1} n_j + \frac{U}{2}\sum_{j=0}^{L-1}n_j(n_j-1)$$
where $J$ is the hopping matrix element, $\mu$ -- the chemical potential, and $U$ -- the interaction strength. We label the lattice sites by $j=0,\dots,L-1$, and use periodic boundary conditions.

First, we load the required packages:

In [34]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import boson_basis_1d # Hilbert space spin basis
import numpy as np # generic math functions

Next, we define the model parameters:

In [35]:
##### define model parameters #####
L=6 # system size
J=1.0 # hopping
U=np.sqrt(2) # interaction
mu=0.0 # chemical potential

In order to construct the Hamiltonian of the BHM, we need to construct the bosonic basis. This is done with the help of the constructor `boson_basis_1d`. The first argument is the chain length, and is required. As an optional argument one cna also specify the number of bosons in the chain `Nb`. 

In [36]:
##### construct single-particle Hamiltonian #####
# define boson basis with 3 states per site L bosons in the lattice
basis = boson_basis_1d(L,Nb=L) # full boson basis
#print(basis)

If needed, we can specify the on-site bosonic Hilbert space dimension, i.e. the number of states per site, using the flag `sps`. 

In [37]:
basis = boson_basis_1d(L,Nb=L,sps=3) # reduced basis, 3 states per site
#print(basis)

Often times, the model under consideration has underlying symmetries. For instance, translation invariance, parity (reflection symmetry), etc. QuSpin allows the user to construct Hamiltonians in symmetry-reduced subspaces. This is done using optional arguments (flags) passed to the basis constructor. 

For instance, if we want to construct the basis in the $k=0$ total momentum sector, we specify this using the flag `kblock=int`. This specifies the many-body momentum of the state via $k=2\pi/L\times\texttt{kblock}$.

Whenever symmetries are present, the `print()` function returns one representative from which one cna obtain al 'missing' states by applying the corresponding symmetry operator. Physically, this representative state stands for the linear combination of vectors in the class.

In [38]:
basis = boson_basis_1d(L,Nb=L,sps=3,kblock=0) # ... and zero momentum sector
#print(basis)

Additionally, the BHM features reflection symmetry around the middle of the chain. This symmetry block-diagonalises the Hamiltonian into two blocks, corresponding to the negative and positive eigenvalue of the parity operator. The corresponding flag is `pblock=+1,-1`.

In [39]:
basis = boson_basis_1d(L,Nb=L,sps=3,kblock=0,pblock=1) # ... + zero momentum and positive parity
#print(basis)

Now that we have constructed the basis in the symmetry-reduced Hilbert space, we can construct the Hamiltonian. It will be hepful to cast it in the fllowing form:

$$H= -J\sum_{j=0}^{L-1}(b^\dagger_{j+1}b_j + \mathrm{h.c.})-\left(\mu+\frac{U}{2}\right)\sum_{j=0}^{L-1} n_j + \frac{U}{2}\sum_{j=0}^{L-1}n_jn_j $$

We start by defining the site-coupling lists

In [40]:
# define site-coupling lists
hop=[[-J,i,(i+1)%L] for i in range(L)] #PBC
interact=[[0.5*U,i,i] for i in range(L)] # U/2 \sum_j n_j n_j
pot=[[-mu-0.5*U,i] for i in range(L)] # -(\mu + U/2) \sum_j j_n

#print(hop)
#print(interact)
#print(pot)

The site coupling lists specify the sites on which the operators act, yet we need to tell QuSpin which operators are to act on these (pairs of) sites. Thus, we need the following operator strings which enter the static and dynamic lists used to define the Hamiltonian. Since the BHM is time-independent, we use an empty dynamic list

In [41]:
# define static and dynamic lists
static=[['+-',hop],['-+',hop],['n',pot],['nn',interact]]
dynamic=[]

Building the Hamiltonian with QuSpin is a one-liner 

In [42]:
# build Hamiltonian
H=hamiltonian(static,dynamic,basis=basis,dtype=np.float64)

Hermiticity check passed!
Symmetry checks passed!
Particle conservation check passed!


We can now diagonalise it, and e.g. calculate the entanglement entropy of the ground state.

In [43]:
# calculate eigensystem
E,V=H.eigh()
E_GS,V_GS=H.eigsh(k=2,which='SA',maxiter=1E10) # only GS
#print("eigenenergies:", E)
#print("GS energy is %0.3f" %(E_GS[0]))
# calculate entanglement entropy per site of GS
subsystem=[i for i in range(L//2)] # sites contained in subsystem
Sent=basis.ent_entropy(V[:,0],sub_sys_A=subsystem)['Sent']/L
#print("GS entanglement per site is %0.3f" %(Sent))
